In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [2]:
def aggregate_velocity(change_rates):
    if len(change_rates) == 0:
        return
    total_change = 0
    isFirst = True
    for rate in change_rates:
        if isFirst:
            isFirst = False
            continue
        total_change += rate
    return total_change/len(change_rates)

def aggregate_direction(directions):
    horizontal_count = 0
    vertical_count = 0
    static_count = 0
    no_direction = 0
    for direction in directions:
        if direction == 'Horizontal':
            horizontal_count += 1
        elif direction == 'Vertical':
            vertical_count += 1
        elif direction == 'Static':
            static_count += 1
        else:
            no_direction += 1
#     print(horizontal_count)
#     print(vertical_count)
#     print(no_direction)
    if static_count > horizontal_count and static_count > vertical_count:
        return 'Static'
    if no_direction > horizontal_count and no_direction > vertical_count:
        return 'No Logo Detectable'
    if horizontal_count > vertical_count:
        return 'Horizontal'
    return 'Vertical'

def get_logo_speed(velocity):
    if velocity == None:
        return 'No Speed'
    if velocity < 10:
        return 'Normal Speed'
    else:
        return 'Abnormal Fast'

In [3]:
def load_logo_detection_yolo_model(model_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)
    return model

In [4]:
# Calculate speed and direction coordinates
def calculate_direction(coordinates, prev_X, prev_Y):
    movement = ''
    rate_of_change = ''
    # print('x = ' + str(coordinates[0][0]) + ' ' + 'y = ' + str(coordinates[0][1]))
    # Focus on a single logo
    newX = coordinates[0][0]
    newY = coordinates[0][1]
    change_in_position_x = abs(newX - prev_X)
    change_in_position_y = abs(newY - prev_Y)
    if change_in_position_x < 1 and change_in_position_y < 1:
        movement = 'Static'
        rate_of_change = 0
    elif change_in_position_y < change_in_position_x:
        movement = 'Horizontal'
        rate_of_change = change_in_position_x
    else:
        movement = 'Vertical'
        rate_of_change = change_in_position_y
    if rate_of_change > 100:
        rate_of_change /= 100
    return newX, newY, movement, rate_of_change

In [5]:
def predict_speed_direction(model, video_path):
    cap = cv2.VideoCapture(video_path)
    total_movement_sets = 10
    frame_limit = 5
    previous_x = 0
    previous_y = 0
    movement_sets = 0
    frame_count = 0
    change_rates = []
    directions = []
    while cap.isOpened() and movement_sets < total_movement_sets:
        ret, frame = cap.read()
        # Make Detections
        results = model(frame)
#         cv2.imshow('YOLO', np.squeeze(results.render()))
        if frame_count == frame_limit:
            # xyxy = xmin, ymin, xmax, ymax, confidence, class
            coordinates = results.xyxy[0]
            detections = len(coordinates)
            if detections != 0:
                newX, newY, direction, change_rate = calculate_direction(coordinates, previous_x, previous_y)
                previous_x = newX
                previous_y = newY
                directions.append(direction)
                change_rates.append(change_rate)
            else:
                print('Unable to Detect Images')
                directions.append('NO IMAGE')
            movement_sets += 1
            frame_count = 0
        frame_count += 1
        # Exit loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
#     print(change_rates)
#     print(directions)
    
    average_velocity = aggregate_velocity(change_rates)
    predicted_direction = aggregate_direction(directions)
    
    return average_velocity, predicted_direction

### Do Prediction on video

In [6]:
yolo_model = load_logo_detection_yolo_model('./TrainedYoloModel/best3.pt')

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\dojh1/.cache\torch\hub\master.zip
YOLOv5  2022-4-3 torch 1.8.2+cu111 CUDA:0 (NVIDIA GeForce GTX 1060 3GB, 3072MiB)

Fusing layers... 
Model summary: 213 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


In [10]:
velocity, direction = predict_speed_direction(yolo_model, './Videos/IMG_5531.mp4')
speed_rating = get_logo_speed(velocity)

print('-----------------------------------')
print('Rate of Change: ' + str(velocity))
print(speed_rating)
print('Movement Direction: ' + direction)

-----------------------------------
Rate of Change: tensor(4.52617, device='cuda:0')
Normal Speed
Movement Direction: Vertical
